# Setting

In [1]:
import sys

sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")

from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_client_v2 as iC
from Clust.clust.ingestion.mongo import mongo_client
from Clust.clust.ML.common import ML_api
from Clust.clust.ML.tool import meta

influxdb_client = iC.InfluxClient(ins.CLUSTDataServer2)
mongodb_client = mongo_client.MongoClient(ins.CLUSTMetaInfo2)

import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

cpu is available.
cpu is available.


# 1. set param from Front End

In [2]:
# model parameter
beatgan_ad_model_info = {
    'input_c': 51,     # the number of feature
    'hidden_c': 32,
    'latent_c': 16
}
# TBD
at_ad_model_info = {
    'k': 3,
    'win_size': 100,
    'input_c': 51,
    'output_c': 51,
    'anomaly_ratio': 0.5
}
# TBD
lstm_vae_ad_info = {
    'input_c': 51,
    'win_size': 100,
    'hidden_size': 30,
    'latent_size': 20
}

# train parameter
train_parameter = {
    'lr': 0.0001,
    'n_epochs': 1,
    'input_size': 51,       # train param?
    'batch_size': 32,
    'device': 'cpu',
}


params1 = {
    'ingestion_param_X': {
        'bucket_name': 'local',
        'ms_name': 'anomaly_detection_SWaT',
        'feature_list': ['FIT101', 'LIT101', 'MV101', 'P101', 'P102', 'AIT201', 'AIT202', 'AIT203', 
                         'FIT201', 'MV201', 'P201', 'P202', 'P203', 'P204', 'P205', 'P206', 
                         'DPIT301', 'FIT301', 'LIT301', 'MV301', 'MV302', 'MV303', 'MV304', 
                         'P301', 'P302', 'AIT401', 'AIT402', 'FIT401', 'LIT401', 'P401', 'P402','P403', 'P404', 
                         'UV401', 'AIT501', 'AIT502', 'AIT503', 'AIT504',
                         'FIT501', 'FIT502', 'FIT503', 'FIT504', 'P501', 'P502', 'PIT501', 'PIT502',
                         'PIT503', 'FIT601', 'P601', 'P602', 'P603']
    },
    'ingestion_param_y': {
        'bucket_name': 'local',
        'ms_name': 'anomaly_detection_SWaT',
        'feature_list': ['attack']
    },
    'data_y_flag': 'true',
    'scaler_param':{
        'scaler_flag': 'none',
        'scale_method': 'none',
        'sclaer_path': 'none'
    },
    'transform_param': {
        'split_mode': 'window_split',
        'data_clean_option': 'none'
    },
    'model_info': {
        'model_purpose': 'anomaly_detection',
        'model_method': 'BeatGAN_ad',       # 'BeatGAN_ad', 'AT_ad', 'LSTM_VAE_ad'
        'model_name': 'None',
        'model_tags': 'tagstest',
        'train_parameter': train_parameter,
        'model_parameter': beatgan_ad_model_info
    }
}

# 2. Backend

In [3]:
# 1. Backend Parameter Setting
params = params1

# parameter tunning
params = ML_api.convert_param_for_backend(params)

In [4]:
# for data preparation
def pre_test_df(test_df):
    test_df['attack'] = test_df['attack'].apply(lambda x : 1 if x > 0 else 0)
    split_index = int(len(test_df) * 0.5)
    
    valid_x = test_df.iloc[:split_index,:].drop(columns=['timestamp','attack'])
    valid_y = test_df.iloc[:split_index,:]['attack']

    test_x = test_df.iloc[split_index:,:].drop(columns=['timestamp','attack'])
    test_y = test_df.iloc[split_index:,:]['attack']
    
    return valid_x, valid_y, test_x, test_y

In [5]:
# 2. Data Preparation from local

# 추후 DB 연동 시 ML_api 의 데이터 준비 형식으로 대체 가능
# train_X_array, train_y_array, val_X_array, val_y_array = AD_api.train_data_preparation(params, influxdb_client)

# scaled data from local
import pandas as pd

# Load data
# from train.csv
df = pd.read_csv('./data/train_ver2.csv')
train_X = df.drop(columns=['timestamp', 'attack'])
train_y = df['attack']

# from test.csv
# val/test set split
test_df = pd.read_csv('./data/test_ver2.csv')
val_X, val_y, _, _ = pre_test_df(test_df)

In [6]:
print(type(train_X))
print(train_X.shape)  # TBD: [seq_len, n_features]

<class 'pandas.core.frame.DataFrame'>
(49680, 51)


In [7]:
# 3. Training
params = ML_api.ML_train(params, train_X, train_y, val_X, val_y)

./Models/Beatgan_ad/anomaly_detection_SWaT_anomaly_detection_BeatGAN_ad_/anomaly_detection_SWaT/model.pkl
Start training model

Current Epoch : 1
Test inference start


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 692/692 [00:29<00:00, 23.74it/s]



best auroc : 0.3463481623247009
[1/1] Train : [loss_d : 0.0599 loss_g : 4.2863] | Valid : [loss_d : 78.5611 loss_g : 2.7762]
Valid AUROC : 0.3463481623247009

Training complete in 11m 51s


In [8]:
print(params)

{'ingestion_param_X': {'bucket_name': 'local', 'ms_name': 'anomaly_detection_SWaT', 'feature_list': ['FIT101', 'LIT101', 'MV101', 'P101', 'P102', 'AIT201', 'AIT202', 'AIT203', 'FIT201', 'MV201', 'P201', 'P202', 'P203', 'P204', 'P205', 'P206', 'DPIT301', 'FIT301', 'LIT301', 'MV301', 'MV302', 'MV303', 'MV304', 'P301', 'P302', 'AIT401', 'AIT402', 'FIT401', 'LIT401', 'P401', 'P402', 'P403', 'P404', 'UV401', 'AIT501', 'AIT502', 'AIT503', 'AIT504', 'FIT501', 'FIT502', 'FIT503', 'FIT504', 'P501', 'P502', 'PIT501', 'PIT502', 'PIT503', 'FIT601', 'P601', 'P602', 'P603']}, 'ingestion_param_y': {'bucket_name': 'local', 'ms_name': 'anomaly_detection_SWaT', 'feature_list': ['attack']}, 'data_y_flag': True, 'scaler_param': {'scaler_flag': None, 'scale_method': None, 'sclaer_path': None}, 'transform_param': {'split_mode': 'window_split', 'data_clean_option': None, 'nan_process_info': {'type': 'num', 'ConsecutiveNanLimit': 10000, 'totalNaNLimit': 100000}, 'max_nan_limit_ratio': 0.5}, 'model_info': {'mo

In [9]:
# save meta
from Clust.clust.ML.tool import meta
meta_file_name = "./meta.json"
#ml_meta.save_model_meta_into_mongodb(mongodb_client, param, 'model','meta')
meta.save_model_meta_into_local(meta_file_name, params)

======== OK ========


200